# 从全连接层到卷积
对于多层感知机或者线性模型都只是适合用于处理表格数据, 但是对于图片数据, 如果利用 `MLP` 进行处理, 那么对于一张图片, 可能对应百万个特征的数据, 从而使得一次训练过程中涉及到的参数过多

## 不变性
对于适合于计算机视觉的神经网络架构, 需要满足下面条件:
1. **平移不变性**: 不管检测对象出现在对象的哪一个位置, 神经网络的前面几层应该对相同的图像区域具有相似反应, 也就是 **"平移不变性"**
2. **局部性:** 神经网络前面几层应该只探索输入图像中的局部区域, 而不在意图像中相隔较远区域的关系, 也就是局部性, 最终可以通过聚合这些局部特征, 从而进行整个图像级别进行预测

## 多层感知机的限制
对于 `MLP` 模型, 假设输入二维图像 $\mathbf{X}$, 隐藏表示 $\mathbf{H}$ 是一个矩阵, 其中可以假设 $X$ 和 $H$ 的形状相同, 不妨使用 $\mathbf{[X]}_{i, j}$ 和 $\mathbf{[H]}_{i, j}$表示输入图像和隐藏中表示位置 $(i, j)$ 处的像素, 注意到隐藏层中的一个像素点需要来自原图像的所有像素加权, 所以对于一个像素点, 权重矩阵大小为 $m * n$, 所以此时需要一个四维矩阵 $\mathbf{W} \in \mathbb{R}^{(m \times n) \times (m \times n)}$, 同时使用 $\mathbf{U} \in \mathbb{R}^{m \times n}$, 所以此时全连接层关系可以表示为:
$$
\begin{aligned}
\mathbf{H}_{i,j} &= [\mathbf{U}]_{i,j} + \sum_{k} \sum_{l} [\mathbf{W}]_{i,j,k,l} [\mathbf{X}]_{k,l} \\
&= [\mathbf{U}]_{i,j} + \sum_{a} \sum_{b} [\mathbf{V}]_{i,j,a,b} [\mathbf{X}]_{i+a,j+b}.
\end{aligned}
$$
其中对于 $(i, j)$ 位置的元素, $V_{i, j}$ 矩阵对应的 $(a, b)$ 位置的元素就是原图像中, $(i+a, j+b)$ 位置的元素的权重

### 平移不变性
由于图像的平移不变性, 所以其实 $\mathbf{V}$ 和 $\mathbf{U}$ 实际上不依赖 $(i, j)$(根据平移性, 对于 $(i,j)$位置的元素, 如果检测对象在输入 $\mathbf{X}$ 中平移, 那么映射到隐藏层中 $\mathbf{H}$ 的表现只能是平移变换), 所以 $[V]_{i, j , a, b} = [\mathbf{V}]_{a, b}$, 并且 $\mathbf{U}$ 是一个常数, 那么可以简化为:
$$
\mathbf{H}_{i,j} = u + \sum_{a} \sum_{b} \mathbf{V}_{a,b} \mathbf{X}_{i+a,j+b}
$$

### 局部性
局部性规定了隐藏层中对于原 $(i, j)$ 位置的像素点的偏移大小限制, 比如限制大小为 $\Delta$, 所以在 $|a| > \Delta, |b| > \Delta$ 的位置, 可以设置 $[\mathbf{V}]_{a, b} = 0$, 所以此时关系可以写成:
$$
\mathbf{H}_{i,j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \mathbf{V}_{a,b} \mathbf{X}_{i+a,j+b}
$$
化简之后, 这就是一个卷积层, 其中 $\mathbf{V}$ 称为**卷积核**或者**滤波器**, 或者称为该卷积层的权重

## 卷积
首先可以回顾一下卷积操作, 比如在数学中, 两个函数 $f, g: \mathbb{R}^d \rightarrow \mathbb{R}$之间的卷积定义如下:
$$
(f * g)(\mathbf{x}) = \int f(\mathbf{z})g(\mathbf{x - z}) d\mathbf{z}
$$
可以理解为把一个函数进行"翻转"(关于 `y` 轴对称)并且移位 $\mathbf{x}$, 之后测量 $f, g$之间的重叠, 在离散情况下:
$$
(f * g)(i) = \sum_{a}f(a)g(i - a)
$$
对于二维张量, 那么 $f$ 的索引 $(a, b)$和 $g$的索引 $(i - a, i - b)$ 上的加权和:
$$
(f * g)(i, j) = \sum_a \sum_b f(a, b)g(i - a, i - b)
$$
可以发现隐藏层的结果其实就是图像和卷积核的的二维卷积

## 卷积层在计算机视觉问题上的应用

### 通道
注意到对于 `RGB` 图像, 一般的图像包含三个通道/三种原色, 图像不是一个二维张量, 而是一个由高度、官渡、颜色组成的三维张量, 包含 1024*1024*3 个像素, 前来嗯个轴与像素的空间位置有关, 而第三个轴体可以看成每一个像素的多维表示, 比如 $X$ 索引为 $[X]_{i, j, k}$, 由此卷积相应地调整为 $[\mathbf{V}]_{a, b, c}$(可以理解成三个方向上的偏移, 也可以把每一个颜色想象成一层)

同时对于隐藏层, 我们可能不仅仅是想要找到输入图像和输出图像之间的特征, 比如我们可能想要一组特征表示(比如一组特征表示纹理, 另外一组特征表示边缘等)

所以为了支持输入和隐藏表示 $\mathbf{H}$ 中的多个通道, 这里可以在 $\mathbf{V}$ 中提那家第四个坐标 $d$ (注意到 $\mathbf{H}$ 中第三个维度不需要表示像素了, 只需要表示特征即可):

$$
[\mathbf{H}]_{i,j,d} = \sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} \sum_{c} [\mathbf{V}]_{a,b,c,d} [\mathbf{X}]_{i+a,j+b,c}$
$$

注意 $d$ 为隐藏层特征数量, 也表示通道数量(特征数量), $c$ 表示每一个色素的颜色值